In [5]:
import pandas as pd
import numpy as np

营养素的库存管理复杂，经手人多，账套混乱，操作原始。核对起来相当痛苦。
现在我来理一下思路。
首先，营养素有住院部和服务部2个部门在同时售卖。
住院部的库存在食堂仓库，需要人工清点。服务部的库存在服务部，需要人工清点。

首先，营养素的管理有两个系统。
## 总系统：百威进销存系统
百威进销存系统记录库存，这个系统减去当天服务部售卖的营养素，但是住院部售卖的营养素需要到月底根据手工统计减去。
### 医院的记账系统 
医院的记账系统只记录住院部的售卖情况。不记录库存。
住院部的工作除我以外还有两人参与：

住院部订单员：下达订单，通知仓库管理员来取货。自己维护一张订单表格

仓库管理员：接到通知时在当日内取货。并计入医院的记账系统
#### 出错可能
食堂管理员少拿错拿营养素，记错帐
#### 开发功能需求
定期核对前一日食堂管理员的医院帐和订单员维护的帐
### 服务部的记账方法 
服务部的帐月末结入百威系统，当月售卖
#### 出错可能
服务部在系统内记账，记漏记错
#### 开发功能需求
记录服务部每日库存差和挂账是否相等

#### 综合需求
昨日百威进销存系统的结存数量 = 仓库库存+服务部库存+服务部挂账+核对后的住院部订单
## 库存管理：

### 入库：
入库由我报数量下订单给采购员，采购员送货后。服务部将其录入。由于服务部有退货可能，所以需在服务部验收后将数量报给我。
### 借调
服务部库存不够时，由我向食堂调货。这时，需在系统内调整库存。
## 保质期管理：
没想好怎么管，各种批号的营养素掺在一起 麻烦炸了先记录下来 功能保留 （概率论）
## 回溯查询：
要有每次查询要有历史记录，可以随时回档。使用git进行版本管理
## 使用场景
每天早上
我：核对订单员和医院记账系统是否对应
系统：读取订单员表格 数据清洗整理 统计出每天每个品卖出几个 去记账系统看
我：准备一张表，填入核对日期，入库量，失效日期，百威结存数量，仓库库存，服务部库存（允许填不变），昨日借调数量，服务部挂账，住院部售卖数量
系统：核对数据，提示出错商品错了几个。并更新明天的表格
我：git add git commit git push



In [4]:
df = pd.read_csv('inventory_count.csv')
df

,RQDB,1,0
0,YZS,2,1
1,YTS,3,1
2,YNS,4,1
3,YQS,4,1
4,YDS,6,2
5,YJS,7,1
6,YMZ,8,1
7,YMS,9,1
